In [32]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression,LinearRegression,SGDRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score,classification_report, confusion_matrix, mean_absolute_error, mean_squared_error,r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor,AdaBoostRegressor, BaggingRegressor
from sklearn.svm import SVR,LinearSVR,NuSVR
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional, GRU, LSTM, RepeatVector, TimeDistributed,Conv1D,MaxPooling1D,ReLU,UpSampling1D,Input,LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from keras.layers.core import Dense
from keras.layers.merge import concatenate
from tensorflow.keras.layers import Average
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
import pywt
from scipy.fft import fft,ifft
from sklearn.preprocessing import OneHotEncoder

In [33]:
data = pd.read_csv('train_FD001.txt', sep = ' ', header = None)
data.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
           dtype='int64')

In [34]:
data_act = data[[f for f in range(0, 26)]]
data_act.columns = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
len(np.unique(data_act['OpSet2']))

13

In [35]:
max_cycles_df = data_act.groupby(["ID"], sort=False)["Cycle"].max().reset_index().rename(columns={"Cycle" : "MaxCycleID"})
max_cycles_df.head()

,ID,MaxCycleID
0,1,192
1,2,287
2,3,179
3,4,189
4,5,269


In [36]:
FD001_df = pd.merge(data_act, max_cycles_df, how="inner", on="ID")
FD001_df["RUL"] = FD001_df["MaxCycleID"] - FD001_df["Cycle"]
FD001_df.head()

,ID,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,...,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21,MaxCycleID,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,192,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,192,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,192,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,192,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,192,187


In [37]:
df_FD001 = FD001_df.copy().drop(columns=["ID", "Cycle", "OpSet1","OpSet2","OpSet3", "MaxCycleID", "RUL"], axis=1)
scaler = MinMaxScaler(feature_range=(0, 1))
# print(df_FD001)
scaled_df_FD001 = pd.DataFrame(scaler.fit_transform(df_FD001.values))
scaled_df_FD001 = FD001_df[["ID", "Cycle", "RUL", "OpSet1","OpSet2","OpSet3"]].join(scaled_df_FD001)
scaled_df_FD001.columns = ["ID", "Cycle","RUL","OpSet1","OpSet2", "OpSet3","SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
print(scaled_df_FD001['OpSet2'].value_counts().idxmax())
print(scaled_df_FD001['OpSet1'].value_counts().idxmax())

-0.0003
0.0


In [38]:
enc_opset1 = OneHotEncoder()
enc_opset2 = OneHotEncoder()
opset1_enc = enc_opset1.fit_transform(scaled_df_FD001['OpSet1'].values.reshape(-1,1)).toarray()
print(opset1_enc.shape)
opset2_enc = enc_opset2.fit_transform(scaled_df_FD001['OpSet2'].values.reshape(-1,1)).toarray()
print(opset2_enc.shape)
print(type(opset1_enc))
print(type(opset2_enc))

(20631, 158)
(20631, 13)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [39]:
scaled_clean_df_FD001 = scaled_df_FD001.drop(columns=["SensorMeasure1", "SensorMeasure5", "SensorMeasure6", "SensorMeasure10",
                                         "SensorMeasure16", "SensorMeasure18", "SensorMeasure19"])
scaled_clean_df_FD001.columns

Index(['ID', 'Cycle', 'RUL', 'OpSet1', 'OpSet2', 'OpSet3', 'SensorMeasure2',
       'SensorMeasure3', 'SensorMeasure4', 'SensorMeasure7', 'SensorMeasure8',
       'SensorMeasure9', 'SensorMeasure11', 'SensorMeasure12',
       'SensorMeasure13', 'SensorMeasure14', 'SensorMeasure15',
       'SensorMeasure17', 'SensorMeasure20', 'SensorMeasure21'],
      dtype='object')

In [40]:
num_col = ['SensorMeasure2',
       'SensorMeasure3', 'SensorMeasure4', 'SensorMeasure7', 'SensorMeasure8',
       'SensorMeasure9', 'SensorMeasure11', 'SensorMeasure12',
       'SensorMeasure13', 'SensorMeasure14', 'SensorMeasure15',
       'SensorMeasure17', 'SensorMeasure20', 'SensorMeasure21']

In [41]:
X_num_tr = scaled_clean_df_FD001[num_col].values
print(X_num_tr.shape)
print(type(X_num_tr))
# X_tr = np.concatenate([X_num_tr,opset1_enc,opset2_enc], axis = 1)
# print(X_tr.shape)
y_tr = scaled_clean_df_FD001['RUL'].values
print(y_tr.shape)

(20631, 14)
<class 'numpy.ndarray'>
(20631,)


In [42]:
def get_seq(in_arr,out_arr,lb):
    x,y = [],[]
    for i in range(in_arr.shape[0]):
        if (i+lb) <= in_arr.shape[0]:
            x.append(in_arr[i:(i+lb),:])
            y.append(out_arr[(i+lb)-1])
    x = np.array(x)
    y = np.array(y)
    return x,y

In [43]:
X_num_tr_win,y_tr1 = get_seq(X_num_tr,y_tr,128)
opset1_enc_win,y_tr2 = get_seq(opset1_enc,y_tr,128)
opset2_enc_win,y_tr3 = get_seq(opset2_enc,y_tr,128)
y_tr1 == y_tr2

array([ True,  True,  True, ...,  True,  True,  True])

In [44]:
def get_wavelet_coeff(inp_ar):
    inp_ar_w = []
    for i in range(inp_ar.shape[0]):
        seq_ar = inp_ar[i,:,:]
        seq_w = np.empty([seq_ar.shape[0],seq_ar.shape[1]])
        for j in range(seq_ar.shape[1]):
            seq_t = seq_ar[:,j]
            (ca5,cd5,cd4,cd3,cd2,cd1) = pywt.wavedec(seq_t, 'haar', level = 5)
            w_f = np.concatenate([ca5,cd5,cd4,cd3,cd2,cd1])
            seq_w[:,j] = w_f
        inp_ar_w.append(seq_w)
    inp_ar_w = np.array(inp_ar_w)
    return inp_ar_w

In [45]:
X_tr_num_w = get_wavelet_coeff(X_num_tr_win)
print(X_tr_num_w.shape)

(20504, 128, 14)


In [46]:
x_train = np.concatenate([X_tr_num_w,opset1_enc_win,opset2_enc_win],axis =2)
print(x_train.shape)
y_train = y_tr1
print(y_train.shape)

(20504, 128, 185)
(20504,)


In [16]:
test_data_FD001 = pd.read_csv('test_FD001.txt', sep = ' ', header = None)
test_data_FD001 = test_data_FD001[[f for f in range(0, 26)]]
test_data_FD001.columns = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
test_data_FD001.head()

,ID,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,...,SensorMeasure12,SensorMeasure13,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


In [17]:
max_cycles_df_test = test_data_FD001.groupby(["ID"], sort=False)["Cycle"].max().reset_index().rename(columns={"Cycle" : "MaxCycleID"})
max_cycles_df_test.head()

,ID,MaxCycleID
0,1,31
1,2,49
2,3,126
3,4,106
4,5,98


In [18]:
y_test_rul_FD001 = pd.read_csv('RUL_FD001.txt', header = None)
y_test_rul_FD001.columns = ['Eng_rul']
y_test_rul_FD001['ID'] = max_cycles_df_test['ID']
y_test_rul_FD001.head()

,Eng_rul,ID
0,112,1
1,98,2
2,69,3
3,82,4
4,91,5


In [19]:
max_cycles_df_test['MaxCycleIDAct'] = y_test_rul_FD001['Eng_rul']+ max_cycles_df_test['MaxCycleID']
max_cycles_df_test.head()

,ID,MaxCycleID,MaxCycleIDAct
0,1,31,143
1,2,49,147
2,3,126,195
3,4,106,188
4,5,98,189


In [20]:
test_df_FD001 = pd.merge(test_data_FD001, max_cycles_df_test, how="inner", on="ID")
test_df_FD001.head()

,ID,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,...,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21,MaxCycleID,MaxCycleIDAct
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,31,143
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,31,143
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,31,143
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,31,143
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,31,143


In [21]:
test_df_FD001["RUL"] = test_df_FD001["MaxCycleIDAct"] - test_df_FD001["Cycle"]
test_df_FD001.head()

,ID,Cycle,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure5,...,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21,MaxCycleID,MaxCycleIDAct,RUL
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,8.4052,0.03,392,2388,100.0,38.86,23.3735,31,143,142
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,8.3803,0.03,393,2388,100.0,39.02,23.3916,31,143,141
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,8.4441,0.03,393,2388,100.0,39.08,23.4166,31,143,140
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,8.3917,0.03,391,2388,100.0,39.00,23.3737,31,143,139
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,8.4031,0.03,390,2388,100.0,38.99,23.4130,31,143,138


In [22]:
te_df_FD001 = test_df_FD001.copy().drop(columns=["ID", "Cycle", "OpSet1", "OpSet2","OpSet3", "MaxCycleID", "MaxCycleIDAct","RUL"], axis=1)
# print(te_df_FD001)
scaled_te_df_FD001 = pd.DataFrame(scaler.transform(te_df_FD001.values))
scaled_te_df_FD001 = test_df_FD001[["ID", "Cycle", "RUL","OpSet1","OpSet2","OpSet3"]].join(scaled_te_df_FD001)
scaled_te_df_FD001.columns = ["ID", "Cycle","RUL", "OpSet1", "OpSet2", "OpSet3","SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
scaled_te_df_FD001.head()

,ID,Cycle,RUL,OpSet1,OpSet2,OpSet3,SensorMeasure1,SensorMeasure2,SensorMeasure3,SensorMeasure4,...,SensorMeasure12,SensorMeasure13,SensorMeasure14,SensorMeasure15,SensorMeasure16,SensorMeasure17,SensorMeasure18,SensorMeasure19,SensorMeasure20,SensorMeasure21
0,1,1,142,0.0023,0.0003,100.0,0.0,0.545181,0.310661,0.269413,...,0.646055,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834
1,1,2,141,-0.0027,-0.0003,100.0,0.0,0.150602,0.379551,0.222316,...,0.739872,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827
2,1,3,140,0.0003,0.0001,100.0,0.0,0.376506,0.346632,0.322248,...,0.699360,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348
3,1,4,139,0.0042,0.0000,100.0,0.0,0.370482,0.285154,0.408001,...,0.573561,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110
4,1,5,138,0.0014,0.0000,100.0,0.0,0.391566,0.352082,0.332039,...,0.737740,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377


In [23]:
unknown_opset1 = [-0.0082, -0.0079, -0.0077, -0.0071, 0.0075, 0.0078]
unknown_opset2 = [0.0007]
for i in range(13096):
    if scaled_te_df_FD001['OpSet1'].iloc[i] in unknown_opset1:
        scaled_te_df_FD001['OpSet1'].iloc[i] = 0.0000
    if scaled_te_df_FD001['OpSet2'].iloc[i] in unknown_opset2:
        scaled_te_df_FD001['OpSet2'].iloc[i] = -0.0003

/Users/shivam.bhardwaj/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:
opset1_enc_te = enc_opset1.transform(scaled_te_df_FD001['OpSet1'].values.reshape(-1,1)).toarray()
opset2_enc_te = enc_opset2.transform(scaled_te_df_FD001['OpSet2'].values.reshape(-1,1)).toarray()
X_te_num = scaled_te_df_FD001[num_col].values
y_te = scaled_te_df_FD001['RUL'].values

In [25]:
opset1_enc_te_win,y_te1 = get_seq(opset1_enc_te,y_te,128)
opset2_enc_te_win,y_te2 = get_seq(opset2_enc_te,y_te,128)
X_te_num_win,y_te3 = get_seq(X_te_num,y_te,128)

In [26]:
X_te_num_w = get_wavelet_coeff(X_te_num_win)

In [27]:
x_test = np.concatenate([X_te_num_w,opset1_enc_te_win,opset2_enc_te_win],axis =2)
print(x_test.shape)
y_test = y_te1
print(y_test.shape)

(12969, 128, 185)
(12969,)


In [28]:
def rul_mdl1(batch_size = None,in_feature = x_train.shape[2], out_f = 1, len_ts = x_train.shape[1]):
    inp = layers.Input(batch_shape= (batch_size, len_ts, x_train.shape[2]), name="Input")
    l1 = layers.LSTM(units = int(x_train.shape[2]/2),return_sequences = True, name = 'Bi-LSTM-1')(inp)
    l1d = layers.Dropout(0.3, name = 'DropOut-1')(l1)
    l2 = layers.LSTM(units = int(x_train.shape[2]/4),return_sequences = True, name = 'Bi-LSTM-2')(l1d)
    l2d = layers.Dropout(0.3, name = 'DropOut-2')(l2)
    l3 = layers.LSTM(units = int(x_train.shape[2]/8),return_sequences = False, name = 'Bi-LSTM-3')(l2d)
    l3d = layers.Dropout(0.3, name = 'Dropout-3')(l3)
    l4 = layers.Dense(units = int(x_train.shape[2]/10), activation = 'relu', name = 'Fully-Connected')(l3d)
    l4d = layers.Dropout(0.3, name = 'Dropout-4')(l4)
    out = layers.Dense(units = out_f, activation = 'linear', name = 'Output')(l4d)
    M = models.Model(inputs = inp, outputs =  out)
    M.compile(loss='mean_squared_error', optimizer='adam')
    return M
Model1 = rul_mdl1()
Model1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 128, 185)]        0         
_________________________________________________________________
Bi-LSTM-1 (LSTM)             (None, 128, 92)           102304    
_________________________________________________________________
DropOut-1 (Dropout)          (None, 128, 92)           0         
_________________________________________________________________
Bi-LSTM-2 (LSTM)             (None, 128, 46)           25576     
_________________________________________________________________
DropOut-2 (Dropout)          (None, 128, 46)           0         
_________________________________________________________________
Bi-LSTM-3 (LSTM)             (None, 23)                6440      
_________________________________________________________________
Dropout-3 (Dropout)          (None, 23)                0     

In [29]:
hist1 = Model1.fit(x_train, y_train, epochs=100, batch_size=4096, verbose=1, validation_split=0.25)
plt.plot(hist1.history['loss'])
plt.plot(hist1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Epoch 1/600
4/4 [==============================] - 55s 12s/step - loss: 15289.1494 - val_loss: 19412.4297
Epoch 2/600
4/4 [==============================] - 43s 11s/step - loss: 15168.5059 - val_loss: 19116.1016
Epoch 3/600
4/4 [==============================] - 42s 11s/step - loss: 14895.8164 - val_loss: 18863.5996
Epoch 4/600
1/4 [======>.......................] - ETA: 33s - loss: 14995.4160

KeyboardInterrupt: 

In [30]:
def rul_mdl2(batch_size = None,in_feature = x_train.shape[2], out_f = 1, len_ts = x_train.shape[1]):
    inp = layers.Input(batch_shape= (batch_size, len_ts, x_train.shape[2]), name="Input")
    l1 = layers.LSTM(units = int(x_train.shape[2]/2),return_sequences = True, name = 'Bi-LSTM1')(inp)
    l1d = layers.Dropout(0.3, name = 'DropOut-1')(l1)
    l1c = layers.Conv1D(filters = int(x_train.shape[2]/2),kernel_size = 32, name = 'Convolution-1')(l1d)
    l1mp = layers.MaxPool1D(pool_size=2, name = 'MaxPool-1')(l1c)
    l2 = layers.LSTM(units = int(x_train.shape[2]/4),return_sequences = True, name = 'Bi-LSTM2')(l1mp)
    l2d = layers.Dropout(0.3, name = 'DropOut-2')(l2)
    l2c = layers.Conv1D(filters = int(x_train.shape[2]/4),kernel_size = 32, name = 'Convolution-2')(l2d)
    l2mp = layers.MaxPooling1D(pool_size=2, name = 'MaxPool-2')(l2c)
    l3 = layers.LSTM(units = int(x_train.shape[2]/10), return_sequences = False)(l2mp)
    l4 = layers.Dense(units = int(x_train.shape[2]/10), activation = 'relu', name = 'Fully-Connected')(l3)
    out = layers.Dense(units = out_f, activation = 'linear', name = 'Output')(l4)
    M = models.Model(inputs = inp, outputs =  out)
    M.compile(loss='mean_squared_error', optimizer='adam')
    return M
Model2 = rul_mdl2()
Model2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 128, 185)]        0         
_________________________________________________________________
Bi-LSTM1 (LSTM)              (None, 128, 92)           102304    
_________________________________________________________________
DropOut-1 (Dropout)          (None, 128, 92)           0         
_________________________________________________________________
Convolution-1 (Conv1D)       (None, 97, 92)            270940    
_________________________________________________________________
MaxPool-1 (MaxPooling1D)     (None, 48, 92)            0         
_________________________________________________________________
Bi-LSTM2 (LSTM)              (None, 48, 46)            25576     
_________________________________________________________________
DropOut-2 (Dropout)          (None, 48, 46)            0   

In [31]:
hist2 = Model2.fit(x_train, y_train, epochs=1000, batch_size=4096, verbose=1, validation_split=0.25)
plt.plot(hist2.history['loss'])
plt.plot(hist2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Epoch 1/6
1/4 [======>.......................] - ETA: 1:07 - loss: 15420.5410

KeyboardInterrupt: 

In [ ]:
rul_pred1,rul_pred2 = Model1.predict(x_test, verbose =1),Model2.predict(x_test, verbose =1)

In [ ]:
y_test = y_test.reshape(-1,1)
res = np.concatenate([y_test, rul_pred1,rul_pred2], axis = 1)

In [ ]:
print(rul_pred1.shape)
print(y_test.shape)
print(res.shape)

In [ ]:
res_df = pd.DataFrame(res, columns=['Actual_RUL', 'RUL_pred1', 'RUL_pred2'])
res_df.to_csv('Result_wavelet_DL_RUL.csv')